# Scope of work

1) Import all necessary libraries and modules.  
2) First we need to obtain inforamtion about all available cars from the first main pages.  
3) Create a SQL database and export there gathered info about cars from the main pages.  
4) Then we will call module dealers_cars to acquire links to every dealer's list of cars from all main pages.  
5) Now we will repeat the same procedure as in the 1st step but to the every dealer's cars list.  
6) Add extracted data to a created SQL database

## 1. Imort of libraries and modules

In [ ]:
import main_pages
import parsing
import marks
import dataframe
import sql_db
import cars_scraper
from datetime import datetime

## 2. Acquring cars' information from the first main pages

Here we call the 1st module 'parsing' to scrap data from the main pages of the website. Then we store all gathered information about cars into corresponding lists.

In [ ]:
# Link to the main webpage
url = 'https://www.autoscout24.com/lst?atype=C&desc=0&sort=standard&source=homepage_search-mask&ustate=N%2CU'

The module **main_pages** collects all URLs of the main pages from the website autoscout24.com

In [ ]:
all_pages = main_pages.pages_urls(url)

With a help of **parsing** module we scrap info about cars from main pages (20).

In [ ]:
start = datetime.now()
cars, characteristics, prices, locations = parsing.cars_info(all_pages)
end = datetime.now()
print('Total time :', end-start)

Here we call the module **marks** in order to extract all existing car marks from the website. Afterwards we will replace
spaces in marks' names into dashes '-'

In [ ]:
marks_menu = marks.all_marks(url)

Here we call the module **dataframe** in order to gather all info about cars into one dataframe

In [ ]:
df = dataframe.df_construct(marks_menu, cars, characteristics, prices, locations)

In [ ]:
df

## 3. Creating a SQL database and exporting parsed data there from the main pages

Here we connect to another module **sql_db**. This module connects to a PostgreSQL database *autoscout*. In this database there is a schema *autoscout* which contains the main table *cars*.

In [ ]:
sql_db.connect(df,'replace')

## 4. Acquring cars' information from the all the car dealers and storing into a SQL database

In [ ]:
start = datetime.now()

for page in all_pages:
    cars_scraper.parser(page, marks_menu)

end = datetime.now()
print('Total time :', end-start)

Total time :1:28:41.573803